<table align="right" width=100%>
    <tr><b></b>
        <td width="25%">
            <img src="https://th.bing.com/th/id/OIP.1lf7rizd_C5U1_ieDXAF2gHaEO?w=319&h=181&c=7&r=0&o=5&dpr=1.1&pid=1.7">
        </td>
        <td>
            <div align="center">
                <font color="#21618C" size=5px>
                  <b>Beijing Multisite Air Quality Machine Learing Model's</b>
                </font>
            </div>
        </td>  
    </tr>
</table>

## Importing the Required Libraries

In [1]:
#Read system file path of data set to load the path
import os
import glob
import math

# import 'Numpy' 
import numpy as np # linear algebra
# import 'Pandas' 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import subpackage of Matplotlib
import matplotlib.pyplot as plt

# import 'Seaborn' 
import seaborn as sns

# to suppress warnings 
from warnings import filterwarnings
filterwarnings('ignore')

# display all columns of the dataframe
pd.options.display.max_columns = None

# display all rows of the dataframe
pd.options.display.max_rows = None
 
# to display the float values upto 6 decimal places     
pd.options.display.float_format = '{:.6f}'.format

# import train-test split 
from sklearn.model_selection import train_test_split

# Selecting the Top 3 cities data w.r.t null values

The data set includes hourly air pollutants data from 12 nationally-controlled air-quality monitoring sites. So we can choose any of the given monitoring site dataset to work upon.
<br><br>
Instead of randomly picking any dataset, It is better to look for the null values. Lesser the null values, better will be the predictions.
<br><br>
We have build a Helper function for finding the best dataset with respect to null values. The idea is to compare the average of col-wise average null values of each multi-site data file.

In [2]:
cd /content/drive/MyDrive/MLESS Project/

/content/drive/MyDrive/MLESS Project


In [3]:
data_path = '/content/drive/MyDrive/MLESS Project/PRSA_Data_20130301-20170228'
# storing names of all datafiles
file_names = [file_name[len(data_path)+1:] for file_name in glob.glob(data_path + '/*.csv')]
for file_name in file_names: print(file_name)

PRSA_Data_Aotizhongxin_20130301-20170228.csv
PRSA_Data_Changping_20130301-20170228.csv
PRSA_Data_Dingling_20130301-20170228.csv
PRSA_Data_Dongsi_20130301-20170228.csv
PRSA_Data_Guanyuan_20130301-20170228.csv
PRSA_Data_Gucheng_20130301-20170228.csv
PRSA_Data_Huairou_20130301-20170228.csv
PRSA_Data_Shunyi_20130301-20170228.csv
PRSA_Data_Tiantan_20130301-20170228.csv
PRSA_Data_Wanliu_20130301-20170228.csv
PRSA_Data_Wanshouxigong_20130301-20170228.csv
PRSA_Data_Nongzhanguan_20130301-20170228.csv


In [4]:
from shutil import copyfile, move

def best_data_file(data_files):
    
    # create directory for storing the dataset
    if not os.path.exists('dataset'):
        os.mkdir('dataset')
    
    # Do the heavy lifting
    best_file = ''
    Min_average = math.inf 
    for file_name in data_files:
        # Reading each datafile in dataframe
        df = pd.read_csv(data_path+'/'+file_name)
        
        # overall average = average of (average of null value col-wise )
        overall_average = df.isna().mean().mean()
        if overall_average < Min_average:
            Min_average = overall_average
            best_file = file_name
        # print(file_name)
        # print(overall_average)
        
        # for visualizing col-wise percentages of null values
        # print(df.isna().mean().round(4)*100)
    
    # copy the best file in the dataset folder
    src = os.path.join(data_path,best_file)
    dst = os.path.join('dataset',best_file)
    move(src,dst)
    print('path created: '+dst)
    return best_file

In [5]:
# test the function
best_file = best_data_file(file_names)

path created: dataset/PRSA_Data_Nongzhanguan_20130301-20170228.csv


In [6]:
file_names.remove("PRSA_Data_Nongzhanguan_20130301-20170228.csv")

In [7]:
best_file = best_data_file(file_names)

path created: dataset/PRSA_Data_Gucheng_20130301-20170228.csv


In [8]:
file_names.remove("PRSA_Data_Gucheng_20130301-20170228.csv")

In [9]:
best_file = best_data_file(file_names)

path created: dataset/PRSA_Data_Wanshouxigong_20130301-20170228.csv


## Data loading and Preprocesing
#### 1) Loading Data

In [10]:
df_1 = pd.read_csv("/content/drive/MyDrive/MLESS Project/dataset/PRSA_Data_Gucheng_20130301-20170228.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/MLESS Project/dataset/PRSA_Data_Nongzhanguan_20130301-20170228.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/MLESS Project/dataset/PRSA_Data_Wanshouxigong_20130301-20170228.csv")
df_1.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,6.000000,18.000000,5.000000,NaN,800.000000,88.000000,0.100000,1021.100000,-18.600000,0.000000,NW,4.400000,Gucheng
1,2,2013,3,1,1,6.000000,15.000000,5.000000,NaN,800.000000,88.000000,-0.300000,1021.500000,-19.000000,0.000000,NW,4.000000,Gucheng
2,3,2013,3,1,2,5.000000,18.000000,NaN,NaN,700.000000,52.000000,-0.700000,1021.500000,-19.800000,0.000000,WNW,4.600000,Gucheng
3,4,2013,3,1,3,6.000000,20.000000,6.000000,NaN,NaN,NaN,-1.000000,1022.700000,-21.200000,0.000000,W,2.800000,Gucheng
4,5,2013,3,1,4,5.000000,17.000000,5.000000,NaN,600.000000,73.000000,-1.300000,1023.000000,-21.400000,0.000000,WNW,3.600000,Gucheng


#### 2) Indexing Data
For time series analysis, Our data should be indexed as Date-Time. For this we will be using pandas DateTimeIndex function. It is important to note that the Time period of our data is from March 1st, 2013 to February 28th, 2017. Lets verify it before proceeding.

In [11]:
# 1st row
df_1[:1]

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,6.000000,18.000000,5.000000,NaN,800.000000,88.000000,0.100000,1021.100000,-18.600000,0.000000,NW,4.400000,Gucheng


In [12]:
# last row
df_1[df_1.shape[0]-1:]

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
35063,35064,2017,2,28,23,12.000000,48.000000,5.000000,48.000000,600.000000,28.000000,6.200000,1010.500000,-13.400000,0.000000,NNE,0.700000,Gucheng


In [13]:
# check data info
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       35064 non-null  int64  
 1   year     35064 non-null  int64  
 2   month    35064 non-null  int64  
 3   day      35064 non-null  int64  
 4   hour     35064 non-null  int64  
 5   PM2.5    34418 non-null  float64
 6   PM10     34683 non-null  float64
 7   SO2      34557 non-null  float64
 8   NO2      34396 non-null  float64
 9   CO       33663 non-null  float64
 10  O3       34335 non-null  float64
 11  TEMP     35013 non-null  float64
 12  PRES     35014 non-null  float64
 13  DEWP     35013 non-null  float64
 14  RAIN     35021 non-null  float64
 15  wd       34905 non-null  object 
 16  WSPM     35022 non-null  float64
 17  station  35064 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 4.8+ MB


Since the columns year, month, day and hour didn't contain any null value, So we can use pandas DateTimeIndex function with start date-time as 2013-03-01 00:00:00 and end date-time as 2017-02-28 23:00:00. As our dataset contains hourly data so we can use freq as 'H' which stands for Hour.

In [14]:
# creating Date-Time index column
df_1['Date-Time'] = pd.date_range(start = '2013-03-01 00:00:00',end = '2017-02-28 23:00:00',freq ='H')
df_2['Date-Time'] = pd.date_range(start = '2013-03-01 00:00:00',end = '2017-02-28 23:00:00',freq ='H')
df_3['Date-Time'] = pd.date_range(start = '2013-03-01 00:00:00',end = '2017-02-28 23:00:00',freq ='H')
# setting index as Date-Time
df_1.set_index('Date-Time', drop = True, inplace = True)
df_2.set_index('Date-Time', drop = True, inplace = True)
df_3.set_index('Date-Time', drop = True, inplace = True)
df_1.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
Date-Time,,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,1,2013,3,1,0,6.000000,18.000000,5.000000,NaN,800.000000,88.000000,0.100000,1021.100000,-18.600000,0.000000,NW,4.400000,Gucheng
2013-03-01 01:00:00,2,2013,3,1,1,6.000000,15.000000,5.000000,NaN,800.000000,88.000000,-0.300000,1021.500000,-19.000000,0.000000,NW,4.000000,Gucheng
2013-03-01 02:00:00,3,2013,3,1,2,5.000000,18.000000,NaN,NaN,700.000000,52.000000,-0.700000,1021.500000,-19.800000,0.000000,WNW,4.600000,Gucheng
2013-03-01 03:00:00,4,2013,3,1,3,6.000000,20.000000,6.000000,NaN,NaN,NaN,-1.000000,1022.700000,-21.200000,0.000000,W,2.800000,Gucheng
2013-03-01 04:00:00,5,2013,3,1,4,5.000000,17.000000,5.000000,NaN,600.000000,73.000000,-1.300000,1023.000000,-21.400000,0.000000,WNW,3.600000,Gucheng


#### 3) Removing unwanted columns

Since we are intrested in time-series analysis of air pollutants We should remove unwanted columns.<br>
The six major air-pollutants are:<br>
1) PM2.5<br>
2) PM10<br>
3) SO2<br>
4) NO2<br>
5) CO<br>
6) O3<br>
Rest of the columns need to be ommited


In [15]:
wanted_col = ['PM2.5','PM10','SO2','NO2','CO','O3']
df_1 = df_1[wanted_col]
df_2 = df_2[wanted_col]
df_3 = df_3[wanted_col]

In [16]:
df_1.head()

,PM2.5,PM10,SO2,NO2,CO,O3
Date-Time,,,,,,
2013-03-01 00:00:00,6.000000,18.000000,5.000000,NaN,800.000000,88.000000
2013-03-01 01:00:00,6.000000,15.000000,5.000000,NaN,800.000000,88.000000
2013-03-01 02:00:00,5.000000,18.000000,NaN,NaN,700.000000,52.000000
2013-03-01 03:00:00,6.000000,20.000000,6.000000,NaN,NaN,NaN
2013-03-01 04:00:00,5.000000,17.000000,5.000000,NaN,600.000000,73.000000


#### 4) Handling Null values

In [17]:
# let us check column-wise null values: before
df_1.isnull().sum()

PM2.5     646
PM10      381
SO2       507
NO2       668
CO       1401
O3        729
dtype: int64

In [18]:
df_2.isnull().sum()

PM2.5     628
PM10      440
SO2       446
NO2       692
CO       1206
O3        506
dtype: int64

In [19]:
df_3.isnull().sum()

PM2.5     696
PM10      484
SO2       669
NO2       754
CO       1297
O3       1078
dtype: int64

In [20]:
df_1['PM2.5'].fillna(df_1['PM2.5'].median(), inplace=True)
df_1['PM10'].fillna(df_1['PM10'].median(), inplace=True)
df_1['SO2'].fillna(df_1['SO2'].median(), inplace=True)
df_1['NO2'].fillna(df_1['NO2'].mean(), inplace=True)
df_1['CO'].fillna(df_1['CO'].median(), inplace=True)
df_1['O3'].fillna(df_1['O3'].median(), inplace=True)


df_2['PM2.5'].fillna(df_2['PM2.5'].median(), inplace=True)
df_2['PM10'].fillna(df_2['PM10'].median(), inplace=True)
df_2['SO2'].fillna(df_2['SO2'].median(), inplace=True)
df_2['NO2'].fillna(df_2['NO2'].mean(), inplace=True)
df_2['CO'].fillna(df_2['CO'].median(), inplace=True)
df_2['O3'].fillna(df_2['O3'].median(), inplace=True)


df_3['PM2.5'].fillna(df_3['PM2.5'].median(), inplace=True)
df_3['PM10'].fillna(df_3['PM10'].median(), inplace=True)
df_3['SO2'].fillna(df_3['SO2'].median(), inplace=True)
df_3['NO2'].fillna(df_3['NO2'].mean(), inplace=True)
df_3['CO'].fillna(df_3['CO'].median(), inplace=True)
df_3['O3'].fillna(df_3['O3'].median(), inplace=True)

In [21]:
# let us check column-wise null values: after
df_1.isnull().sum()

PM2.5    0
PM10     0
SO2      0
NO2      0
CO       0
O3       0
dtype: int64

In [22]:
df_2.isnull().sum()

PM2.5    0
PM10     0
SO2      0
NO2      0
CO       0
O3       0
dtype: int64

In [23]:
df_3.isnull().sum()

PM2.5    0
PM10     0
SO2      0
NO2      0
CO       0
O3       0
dtype: int64

Now we will create six data frames for the pollutants PM2.5, PM10, SO2, NO2, CO and O3 for Time-Series analysis.

In [24]:
def give_columnwise_dataset(cleaned_df):
  # creating data-frame for PM2.5
  df_PM25 = cleaned_df[['PM2.5']]
  # creating data-frame for PM10
  df_PM10 = cleaned_df[['PM10']]
  # creating data-frame for SO2
  df_SO2 = cleaned_df[['SO2']]
  # creating data-frame for NO2
  df_NO2 = cleaned_df[['NO2']]
  # creating data-frame for CO
  df_CO = cleaned_df[['CO']]
  # creating data-frame for O3
  df_O3 = cleaned_df[['O3']]

  return df_PM25, df_PM10, df_SO2, df_NO2, df_CO, df_O3

df_1_PM25, df_1_PM10, df_1_SO2, df_1_NO2, df_1_CO, df_1_O3 = give_columnwise_dataset(df_1)
df_2_PM25, df_2_PM10, df_2_SO2, df_2_NO2, df_2_CO, df_2_O3 = give_columnwise_dataset(df_2)
df_3_PM25, df_3_PM10, df_3_SO2, df_3_NO2, df_3_CO, df_3_O3 = give_columnwise_dataset(df_3)

 #### Resampling data to 'Daily' and 'Monthly' from 'Hourly'
 Since we are going to make a long time series its better to resample our data from Hourly to Daily and Monthly.
 We will use pandas time series tools to resample points at frequency like every day ('D') or start month ('MS'), etc.

In [25]:
# function for resampling each dataframe based on frequency
def resample_data(dfs, freq = 'D'):
    '''
        dfs: list of dataframes
        freq: frequency for resampling data, 'D' for Day
        return: list of resampled dataframes
        
        Note: 1) if freq is 'D' then resampling is done by taking average of hours.
              2) if freq is 'MS' then resampling is done by taking monthly average
                 of maximum values in a day
    
    '''
    if freq == 'D':
        return [each_df.resample(freq).mean() for each_df in dfs] 
    else:
        return [each_df.resample('D').max().resample(freq).mean() for each_df in dfs]

In [26]:
df_1_list = [df_1_PM25, df_1_PM10, df_1_SO2, df_1_NO2, df_1_CO, df_1_O3]
df_2_list = [df_2_PM25, df_2_PM10, df_2_SO2, df_2_NO2, df_2_CO, df_2_O3]
df_3_list = [df_3_PM25, df_3_PM10, df_3_SO2, df_3_NO2, df_3_CO, df_3_O3]
# Resampling dataframes by daily
df_1_PM25, df_1_PM10, df_1_SO2, df_1_NO2, df_1_CO, df_1_O3 = resample_data(df_1_list)
df_2_PM25, df_2_PM10, df_2_SO2, df_2_NO2, df_2_CO, df_2_O3 = resample_data(df_2_list)
df_3_PM25, df_3_PM10, df_3_SO2, df_3_NO2, df_3_CO, df_3_O3 = resample_data(df_3_list)

# Resampling dataframes by monthly
dfm_1_PM25, dfm_1_PM10, dfm_1_SO2, dfm_1_NO2, dfm_1_CO, dfm_1_O3 = resample_data(df_1_list, freq = 'MS')
dfm_2_PM25, dfm_2_PM10, dfm_2_SO2, dfm_2_NO2, dfm_2_CO, dfm_2_O3 = resample_data(df_2_list, freq = 'MS')
dfm_3_PM25, dfm_3_PM10, dfm_3_SO2, dfm_3_NO2, dfm_3_CO, dfm_3_O3 = resample_data(df_3_list, freq = 'MS')

# updating df_list of daily pollutants
df_1_list = [df_1_PM25, df_1_PM10, df_1_SO2, df_1_NO2, df_1_CO, df_1_O3]
df_2_list = [df_2_PM25, df_2_PM10, df_2_SO2, df_2_NO2, df_2_CO, df_2_O3]
df_3_list = [df_3_PM25, df_3_PM10, df_3_SO2, df_3_NO2, df_3_CO, df_3_O3]

# creating dfm_list for monthly pollutants
dfm_1_list = [dfm_1_PM25, dfm_1_PM10, dfm_1_SO2, dfm_1_NO2, dfm_1_CO, dfm_1_O3]
dfm_2_list = [dfm_2_PM25, dfm_2_PM10, dfm_2_SO2, dfm_2_NO2, dfm_2_CO, dfm_2_O3]
dfm_3_list = [dfm_3_PM25, dfm_3_PM10, dfm_3_SO2, dfm_3_NO2, dfm_3_CO, dfm_3_O3]

In [27]:
# lets us see the change in frequency from Hourly to Daily
df_1_PM25.head()

,PM2.5
Date-Time,
2013-03-01,8.625000
2013-03-02,36.916667
2013-03-03,80.333333
2013-03-04,26.500000
2013-03-05,136.708333


In [28]:
# lets us see the change in frequency from Hourly to Monthly
dfm_1_PM25.head()

,PM2.5
Date-Time,
2013-03-01,188.129032
2013-04-01,123.966667
2013-05-01,140.709677
2013-06-01,170.533333
2013-07-01,114.516129


### Train-Test Split

#### For Daily data
We can slice our daily data from March-1-2013 to February-29-2016(note 2016 is a leap year) to Train set So that we could use remaining one year data for testing.<br>
#### For Monthly data
We can slice our monthly data from March-2013 to February-2016 to Train set and remaining one year data for testing.<br>
Let us see what would be the proportion of our train-test data.

In [29]:
# test size from March-1-2016 to Feb-28-2017 / train size from March-1-2013 to Feb-29-2016
df_1_PM25['2016-03-01':].size / df_1_PM25[:'2016-02-29'].size

0.333029197080292

So, the Train and Test proportion will roughly be 2/3 and 1/3 respectively. Let's slice our data to create Train and Test datasets.

In [30]:
# function for train-test split
def train_test_split(dfs, train_end_index = '2016-02-29', test_start_index = '2016-03-01', freq = 'D'):
    '''
        dfs: list of dataframes
        train_end_index: last index for train set
        test_start_index: starting index for test set
        return:  list of tuples containing train-test sets
                 Eg: [(train_x1,test_y1), (train_x2, test_y2), .....]
        
        Note: In slicing DateTimeIndex both start and end index are inclusive
    '''
    if freq == 'M':
        test_start_index = '2016-03'
        train_end_index = '2016-02'
        
    return [(each_df[:train_end_index], each_df[test_start_index:]) for each_df in dfs]
        
 

In [31]:
# splitting daily data of each pollutants into train and test
dtrain_test_list_1 = train_test_split(df_1_list)
dtrain_test_list_2 = train_test_split(df_2_list)
dtrain_test_list_3 = train_test_split(df_3_list)

# splitting monthly data of each pollutants into train and test
mtrain_test_list_1 = train_test_split(dfm_1_list)
mtrain_test_list_2 = train_test_split(dfm_2_list)
mtrain_test_list_3 = train_test_split(dfm_3_list)

Lets Test our train_test_split function.
<br><br><b> Testing split for daily data</b>

In [32]:
# lets see the shape of dtrain_PM25 and dtest_PM25 (daily data)
dtrain_1_PM25 = dtrain_test_list_1[0][0]
dtest_1_PM25 = dtrain_test_list_1[0][1]
dtrain_1_PM25.shape , dtest_1_PM25.shape

((1096, 1), (365, 1))

In [33]:
# lets see the first and last row of dtrain_PM25 (daily data)
print('dtrain_PM25', dtrain_1_PM25.head(1), dtrain_1_PM25.tail(1), sep='\n')
print('------------------------')

# lets see the first and last row of dtest_PM25 (daily data)
print('dtest_PM25', dtest_1_PM25.head(1), dtest_1_PM25.tail(1), sep='\n')

dtrain_PM25
              PM2.5
Date-Time          
2013-03-01 8.625000
               PM2.5
Date-Time           
2016-02-29 42.000000
------------------------
dtest_PM25
                PM2.5
Date-Time            
2016-03-01 128.375000
               PM2.5
Date-Time           
2017-02-28 13.708333


<b> Testing split for monthly data</b>

In [34]:
# lets see the shape of mtrain_PM25 and mtest_PM25 (monthly data)
mtrain_1_PM25 = mtrain_test_list_1[0][0]
mtest_1_PM25 = mtrain_test_list_1[0][1]
mtrain_1_PM25.shape , mtest_1_PM25.shape

((36, 1), (12, 1))

In [35]:
# lets see the first and last row of dtrain_PM25 (daily data)
print('mtrain_PM25', mtrain_1_PM25.head(1), mtrain_1_PM25.tail(1), sep='\n')
print('------------------------')

# lets see the first and last row of dtest_PM25 (daily data)
print('mtest_PM25', mtest_1_PM25.head(1), mtest_1_PM25.tail(1), sep='\n')


mtrain_PM25
                PM2.5
Date-Time            
2013-03-01 188.129032
                PM2.5
Date-Time            
2016-02-01 108.137931
------------------------
mtest_PM25
                PM2.5
Date-Time            
2016-03-01 162.709677
                PM2.5
Date-Time            
2017-02-01 143.357143


As we can see, our train_test_split function worked as expected. Now we can save our Train and Test data locally. Lets create two directories daily and monthly inside dataset folder.

In [36]:
# creating daily and monthly directory
if not os.path.exists('/content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily'): os.mkdir('/content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily')
if not os.path.exists('/content/drive/MyDrive/MLESS Project/dataset/Gucheng/monthly'): os.mkdir('/content/drive/MyDrive/MLESS Project/dataset/Gucheng/monthly')

if not os.path.exists('/content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily'): os.mkdir('/content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily')
if not os.path.exists('/content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/monthly'): os.mkdir('/content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/monthly')

if not os.path.exists('/content/drive/MyDrive/MLESS Project/dataset/Wanshouxigong/daily'): os.mkdir('/content/drive/MyDrive/MLESS Project/dataset/Wanshouxigong/daily')
if not os.path.exists('/content/drive/MyDrive/MLESS Project/dataset/Wanshouxigong/monthly'): os.mkdir('/content/drive/MyDrive/MLESS Project/dataset/Wanshouxigong/monthly')

In [37]:
# function for saving train-test data
def save_train_test_data(train_test_list, train_dir, test_dir):
    '''
        train_test_list = 
        [(train_PM25, test_PM25), (train_PM10, test_PM10), (train_SO2, test_SO2),\
        (train_NO2, test_NO2), (train_CO, test_CO), (train_O3, test_O3)] 

    '''
    # save train-test data
    for each_train_test in train_test_list:
        # fetch each train-test file name
        filename = each_train_test[0].columns.values[0]

        # saving each train file    Note: It's important to reset the index in order to add Date-Time index as column
        (each_train_test[0].reset_index()).to_csv(os.path.join(train_dir, 'train_'+filename+'.csv'), index = False)

        # saving each test file
        (each_train_test[1].reset_index()).to_csv(os.path.join(test_dir, 'test_'+filename+'.csv'), index = False)

        print('path created ',train_dir +'/train_'+filename+'.csv')
        print('path created ',test_dir+'/test_'+filename+'.csv')
    print()

In [38]:
# define train path for daily data
dtrain_1_dir = '/content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/train'
dtrain_2_dir = '/content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/train'
dtrain_3_dir = '/content/drive/MyDrive/MLESS Project/dataset/Wanshouxigong/daily/train'

# define test path for daily data
dtest_1_dir = '/content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/test'
dtest_2_dir = '/content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/test'
dtest_3_dir = '/content/drive/MyDrive/MLESS Project/dataset/Wanshouxigong/daily/test'

# create Train directory inside dataset/daily
if not os.path.exists(dtrain_1_dir): os.mkdir(dtrain_1_dir)
if not os.path.exists(dtrain_2_dir): os.mkdir(dtrain_2_dir)
if not os.path.exists(dtrain_3_dir): os.mkdir(dtrain_3_dir)
# create Test directory inside dataset/daily
if not os.path.exists(dtest_1_dir): os.mkdir(dtest_1_dir)
if not os.path.exists(dtest_2_dir): os.mkdir(dtest_2_dir)
if not os.path.exists(dtest_3_dir): os.mkdir(dtest_3_dir)

# define train path for monthly data
mtrain_1_dir = '/content/drive/MyDrive/MLESS Project/dataset/Gucheng/monthly/train'
mtrain_2_dir = '/content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/monthly/train'
mtrain_3_dir = '/content/drive/MyDrive/MLESS Project/dataset/Wanshouxigong/monthly/train'
# define test path for monthly data
mtest_1_dir = '/content/drive/MyDrive/MLESS Project/dataset/Gucheng/monthly/test'
mtest_2_dir = '/content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/monthly/test'
mtest_3_dir = '/content/drive/MyDrive/MLESS Project/dataset/Wanshouxigong/monthly/test'
# create Train directory inside dataset/monthly
if not os.path.exists(mtrain_1_dir): os.mkdir(mtrain_1_dir)
if not os.path.exists(mtrain_2_dir): os.mkdir(mtrain_2_dir)
if not os.path.exists(mtrain_3_dir): os.mkdir(mtrain_3_dir)
# create Test directory inside dataset/monthly
if not os.path.exists(mtest_1_dir): os.mkdir(mtest_1_dir)
if not os.path.exists(mtest_2_dir): os.mkdir(mtest_2_dir)
if not os.path.exists(mtest_3_dir): os.mkdir(mtest_3_dir)

In [39]:
# save daily data
save_train_test_data(dtrain_test_list_1, dtrain_1_dir, dtest_1_dir)
# save monthly data
save_train_test_data(mtrain_test_list_1, mtrain_1_dir, mtest_1_dir)

path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/train/train_PM2.5.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/test/test_PM2.5.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/train/train_PM10.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/test/test_PM10.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/train/train_SO2.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/test/test_SO2.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/train/train_NO2.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/test/test_NO2.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/train/train_CO.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/test/test_CO.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/tra

In [40]:
# save daily data
save_train_test_data(dtrain_test_list_2, dtrain_2_dir, dtest_2_dir)
# save monthly data
save_train_test_data(mtrain_test_list_2, mtrain_2_dir, mtest_2_dir)

# save daily data
save_train_test_data(dtrain_test_list_3, dtrain_3_dir, dtest_3_dir)
# save monthly data
save_train_test_data(mtrain_test_list_3, mtrain_3_dir, mtest_3_dir)

path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/train/train_PM2.5.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/test/test_PM2.5.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/train/train_PM10.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/test/test_PM10.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/train/train_SO2.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/test/test_SO2.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/train/train_NO2.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/test/test_NO2.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/train/train_CO.csv
path created  /content/drive/MyDrive/MLESS Project/dataset/Nongzhanguan/daily/test/test_CO.csv
path created  /content/dri

In [41]:
# let's test one of the file from daily directory
pd.read_csv('/content/drive/MyDrive/MLESS Project/dataset/Gucheng/daily/train/train_PM2.5.csv').head()

,Date-Time,PM2.5
0,2013-03-01,8.625000
1,2013-03-02,36.916667
2,2013-03-03,80.333333
3,2013-03-04,26.500000
4,2013-03-05,136.708333


In [42]:
# let's test one of the file from monthly directory
pd.read_csv('/content/drive/MyDrive/MLESS Project/dataset/Gucheng/monthly/train/train_PM2.5.csv').head()

,Date-Time,PM2.5
0,2013-03-01,188.129032
1,2013-04-01,123.966667
2,2013-05-01,140.709677
3,2013-06-01,170.533333
4,2013-07-01,114.516129


In next part we will build and fit models with our train-test datasets.